# REST Interact With `adk web`

Make sure you have started `adk web` locally.

In [1]:
!which python

/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/.venv/bin/python


In [1]:
import requests
from config import ADK_SESSION_CREATE_URL, ADK_RUN_SSE_URL, ADK_RUN_URL

In [2]:
ADK_SESSION_CREATE_URL, ADK_RUN_SSE_URL, ADK_RUN_URL

('http://localhost:8000/apps/document_agent/users/user/sessions',
 'http://localhost:8000/run_sse',
 'http://localhost:8000/run')

## Get A Session

In [3]:
# override for testing
ADK_SESSION_CREATE_URL = 'http://localhost:8000/apps'

In [4]:
agent_name = 'document_agent'
user_id = 'u_123'
session_id = 's_123'

In [5]:
ADK_SESSION_URL = f"{ADK_SESSION_CREATE_URL}/{agent_name}/users/{user_id}/sessions/{session_id}"
ADK_SESSION_URL

'http://localhost:8000/apps/document_agent/users/u_123/sessions/s_123'

In [6]:
session_payload = {}
response = requests.post(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [],
 'lastUpdateTime': 1749237712.3047004}

In [8]:
SERVER_SESSION_ID = response_data['id']
SERVER_SESSION_ID

's_123'

## Interact With Session

In [9]:
message_parts = []
init_message = {'text': 'Hello'}
message_parts.append(init_message)

In [10]:
payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [11]:
response = requests.post(ADK_RUN_URL, json=payload, timeout=180)
response.raise_for_status()
response_data = response.json()
response_data

[{'content': {'parts': [{'text': 'Hello! To get started, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) for the document you would like me to process.\n'}],
   'role': 'model'},
  'usageMetadata': {'candidatesTokenCount': 36,
   'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 36}],
   'promptTokenCount': 3470,
   'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3470}],
   'totalTokenCount': 3506,
   'trafficType': 'ON_DEMAND'},
  'invocationId': 'e-ac430462-e524-452a-9d5c-a50cd11b0b78',
  'author': 'document_agent',
  'actions': {'stateDelta': {},
   'artifactDelta': {},
   'requestedAuthConfigs': {}},
  'id': 'trPprm4w',
  'timestamp': 1749237747.11849}]

In [12]:
def parse_response(response_data):
    final_text = "Error: No final response from assistant found."
    if isinstance(response_data, list):
        for message in reversed(response_data):
            content = message.get('content', {})
            if isinstance(content, dict) and content.get("role") == "model":
                parts = content.get('parts', [])
                if parts and isinstance(parts, list):
                    first_part = parts[0]
                    if isinstance(first_part, dict):
                        text_part = first_part.get("text")
                        if text_part:
                            final_text = text_part
                        else:
                            final_text = f"Agent returned a structured response: `{json.dumps(first_part)}`"
                    else:
                        final_text = f"Agent returned an unknown part: `{str(first_part)}`"
                    break
    return final_text

parse_response(response_data)

'Hello! To get started, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) for the document you would like me to process.\n'

## Add To Interaction

In [13]:
message_parts = []
init_message = {'text': 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'}
message_parts.append(init_message)

payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [14]:
response = requests.post(ADK_RUN_URL, json=payload, timeout=180)
response.raise_for_status()
response_data = response.json()
response_data

[{'content': {'parts': [{'functionCall': {'id': 'adk-12ec97b3-1dcc-44d3-b06d-a2fe9b6cdf49',
      'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf',
       'gcs_bucket': 'statmike-mlops-349915'},
      'name': 'get_gcs_file'}}],
   'role': 'model'},
  'usageMetadata': {'candidatesTokenCount': 56,
   'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 56}],
   'promptTokenCount': 3556,
   'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3556}],
   'totalTokenCount': 3612,
   'trafficType': 'ON_DEMAND'},
  'invocationId': 'e-0257ac2a-944b-4fbf-8b96-7148c3853f1b',
  'author': 'document_agent',
  'actions': {'stateDelta': {},
   'artifactDelta': {},
   'requestedAuthConfigs': {}},
  'longRunningToolIds': [],
  'id': 'YqmjDmRF',
  'timestamp': 1749237760.738788},
 {'content': {'parts': [{'functionResponse': {'id': 'adk-12ec97b3-1dcc-44d3-b06d-a2fe9b6cdf49',
      'name': 'get_gcs_file',
      'r

In [15]:
parse_response(response_data)

'Okay, I have loaded the file `vendor_2_invoice_10.pdf` from the GCS bucket. The artifact key for this document is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`.\n'

## Fetch A Session

In [16]:
session_payload = {}
response = requests.get(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [{'content': {'parts': [{'text': 'Hello'}], 'role': 'user'},
   'invocationId': 'e-ac430462-e524-452a-9d5c-a50cd11b0b78',
   'author': 'user',
   'actions': {'stateDelta': {},
    'artifactDelta': {},
    'requestedAuthConfigs': {}},
   'id': 'ogFZi3nX',
   'timestamp': 1749237747.117274},
  {'content': {'parts': [{'text': 'Hello! To get started, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) for the document you would like me to process.\n'}],
    'role': 'model'},
   'usageMetadata': {'candidatesTokenCount': 36,
    'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 36}],
    'promptTokenCount': 3470,
    'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3470}],
    'totalTokenCount': 3506,
    'trafficType': 'ON_DEMAND'},
   'invocationId': 'e-ac430462-e524-452a-9d5c-a50cd11b0b78',
   'author': 'document_agent',
   'actions': {'stateDelta': {},
 

---
### NOTES

- [ ] If artifact_delta then can i retrieve the artifact locally?
- [ ] How can i send a file as inline data - see app implementation for example
- [X] can i retrieve the entire session from the URL?